## Import libraries

In [1]:
import sys
sys.path.insert(0, './haqs_api')

import haqs_api

%load_ext autoreload
%autoreload 2

## Data addresses

1. **Stations request** http://api.gios.gov.pl/pjp-api/rest/station/findAll
2. **Sensors request** http://api.gios.gov.pl/pjp-api/rest/station/sensors/{stationId}
3. **Data request** http://api.gios.gov.pl/pjp-api/rest/data/getData/{sensorId}
4. **AQ index request** http://api.gios.gov.pl/pjp-api/rest/aqindex/getIndex/{stationId}

# Geopandas part

## Get Stations from GIOŚ API

In [3]:
stations_json = haqs_api.get_stations()
print("Number of available stations: ", len(stations))

Number of available stations:  150


In [4]:
stations_df = haqs_api.create_stations_gdf(stations_json)
stations_df.head()

,station_id,geometry
0,114,POINT (17.141125 51.115933)
1,117,POINT (17.02925 51.129378)
2,129,POINT (17.012689 51.086225)
3,52,POINT (16.180513 51.204503)
4,109,POINT (16.269677 50.768729)


## Plot Stations on map

In [5]:
haqs_api.create_stations_map(stations_json)

## Get list of Sensors for each Station

In [6]:
sensors_df = haqs_api.create_sensors_df(stations_json)
sensors_df.head()

,station_id,sensor_id,parameter
0,114,642,NO2
1,114,644,O3
2,117,660,CO
3,117,14395,PM10
4,117,658,C6H6


## Get Sensors readings from GIOŚ API

In [8]:
sensors_df = haqs_api.get_latest_sensors_readings(sensors_df)
sensors_df.head()

,station_id,sensor_id,parameter,data
0,114,642,NO2,6.43335
1,114,644,O3,99.86060
2,117,660,CO,356.15900
3,117,14395,PM10,11.50140
4,117,658,C6H6,0.00034


In [7]:
available_parameters = haqs_api.get_available_parameters(sensors_df)
available_parameters

['NO2', 'O3', 'CO', 'PM10', 'C6H6', 'PM2.5', 'SO2']

## Visualize readings

In [47]:
params_df = haqs_api.get_param_df(stations_df, sensors_df, 'SO2')
haqs_api.show_readings_map(params_df)

Loading BokehJS ...

# Connect with database

In [156]:
conn = haqs_api.connect_with_db()

Successfully conected with DataBase!


In [155]:
haqs_api.close_db_connection(conn)

Connection has been closed!


## Create PostGIS Extension

In [157]:
haqs_api.create_postgis_extension(conn)

SAVEPOINT created!
extension "postgis" already exists

Rollback to SAVEPOINT!
SAVEPOINT released!


## <font color="blue">Station Table</font>

### Create Stations Table

In [158]:
haqs_api.create_stations_table(conn)

SAVEPOINT created!
relation "stations" already exists

Rollback to SAVEPOINT!
SAVEPOINT released!


### Insert Stations into DataBase

1. Using text 
> <i>**INSERT INTO**</i> stations(station_id, geom) **VALUES**(2, ST_GeomFromText('POINT(-71.060316 48.432044)', EPSG));
2. Using lognitude and latitude
> <i>**INSERT INTO**</i> stations(station_id, geom) **VALUES**(2, ST_SetSRID(ST_MakePoint(lon, lat), EPSG));

In [151]:
stations_json = haqs_api.get_stations()

In [161]:
stations_json[0]

{'id': 114,
 'stationName': 'Wrocław - Bartnicza',
 'gegrLat': '51.115933',
 'gegrLon': '17.141125',
 'city': {'id': 1064,
  'name': 'Wrocław',
  'commune': {'communeName': 'Wrocław',
   'districtName': 'Wrocław',
   'provinceName': 'DOLNOŚLĄSKIE'}},
 'addressStreet': 'ul. Bartnicza'}

In [160]:
# iterate over stations
for station in stations_json:
    
    station_id = station["id"]
    station_lon = station['gegrLon']
    station_lat = station['gegrLat']
    
    try:
        haqs_api.db_insert_station(conn, station_id, station_lon, station_lat)
    except Exception as e:
        print(e)
        print ("Cannot execute insertion for Station ID: {}".format(station_id))

SAVEPOINT created!
duplicate key value violates unique constraint "stations_pkey"
DETAIL:  Key (station_id)=(114) already exists.

Rollback to SAVEPOINT!
SAVEPOINT released!
SAVEPOINT created!
duplicate key value violates unique constraint "stations_pkey"
DETAIL:  Key (station_id)=(117) already exists.

Rollback to SAVEPOINT!
SAVEPOINT released!
SAVEPOINT created!
duplicate key value violates unique constraint "stations_pkey"
DETAIL:  Key (station_id)=(129) already exists.

Rollback to SAVEPOINT!
SAVEPOINT released!
SAVEPOINT created!
duplicate key value violates unique constraint "stations_pkey"
DETAIL:  Key (station_id)=(52) already exists.

Rollback to SAVEPOINT!
SAVEPOINT released!
SAVEPOINT created!
duplicate key value violates unique constraint "stations_pkey"
DETAIL:  Key (station_id)=(109) already exists.

Rollback to SAVEPOINT!
SAVEPOINT released!
SAVEPOINT created!
duplicate key value violates unique constraint "stations_pkey"
DETAIL:  Key (station_id)=(11) already exists.

R

SAVEPOINT created!
duplicate key value violates unique constraint "stations_pkey"
DETAIL:  Key (station_id)=(9593) already exists.

Rollback to SAVEPOINT!
SAVEPOINT released!
SAVEPOINT created!
duplicate key value violates unique constraint "stations_pkey"
DETAIL:  Key (station_id)=(10874) already exists.

Rollback to SAVEPOINT!
SAVEPOINT released!
SAVEPOINT created!
duplicate key value violates unique constraint "stations_pkey"
DETAIL:  Key (station_id)=(361) already exists.

Rollback to SAVEPOINT!
SAVEPOINT released!
SAVEPOINT created!
duplicate key value violates unique constraint "stations_pkey"
DETAIL:  Key (station_id)=(387) already exists.

Rollback to SAVEPOINT!
SAVEPOINT released!
SAVEPOINT created!
duplicate key value violates unique constraint "stations_pkey"
DETAIL:  Key (station_id)=(374) already exists.

Rollback to SAVEPOINT!
SAVEPOINT released!
SAVEPOINT created!
duplicate key value violates unique constraint "stations_pkey"
DETAIL:  Key (station_id)=(376) already exist

### Show Insertions

In [162]:
sql = "SELECT * FROM public.stations"

cur = conn.cursor()
cur.execute(sql)
cur.fetchall()

[(114, '0101000020E6100000E3A59BC4202431407AC37DE4D68E4940'),
 (117, '0101000020E6100000736891ED7C073140346953758F904940'),
 (129, '0101000020E6100000DA0418963F0331407E8CB96B098B4940'),
 (52, '0101000020E6100000BB809719362E30408A7780272D9A4940'),
 (109, '0101000020E6100000C07B478D094530404F3E3DB665624940'),
 (11, '0101000020E6100000C746205ED79F2E4074B515FBCB744940'),
 (14, '0101000020E6100000F4346090F4E12D400EA0DFF76F7C4940'),
 (16, '0101000020E61000003D2CD49AE6A53040F6798CF2CC5D4940'),
 (38, '0101000020E61000000FB4024356A73040C005D9B27C374940'),
 (67, '0101000020E6100000C9570229B18330400C3F389F3A4A4940'),
 (70, '0101000020E6100000DE3EABCC944A3140E9B81AD995784940'),
 (74, '0101000020E6100000A71FD4450ADD2E403F1D8F19A8A84940'),
 (84, '0101000020E610000070438CD7BC7A2F402C11A8FE415E4940'),
 (132, '0101000020E6100000FAB7CB7EDDD1304036CD3B4ED14B4940'),
 (134, '0101000020E6100000234A7B832F042E406E4E250340934940'),
 (9153, '0101000020E6100000A81C93C5FD872F40280B5F5FEB744940'),
 (10007, '010100

### Create Stations DataFrame from Stations Table

In [53]:
sql = "SELECT * FROM public.stations"

stations_df = gpd.read_postgis(sql, conn, geom_col='geom')

In [54]:
stations_df.head()

,station_id,geom
0,114,POINT (17.141125 51.115933)
1,117,POINT (17.02925 51.129378)
2,129,POINT (17.012689 51.086225)
3,52,POINT (16.180513 51.204503)
4,109,POINT (16.269677 50.768729)


## <font color="blue">Sensors Table</font>

### Create Sensors Table

In [32]:
sql = """
    CREATE TABLE public.sensors 
    (
        sensor_id INTEGER PRIMARY KEY,
        sensor_parameter VARCHAR(10) NOT NULL,
        station_id INTEGER REFERENCES stations (station_id)
    );
"""
    
cur = conn.cursor()
cur.execute(sql)

### Insert Sensors into DataBase

> <i>**INSERT INTO**</i> sensors(sensor_id, sensor_parameter, station_id) **VALUES** (2, 'PM10', 114));

In [33]:
def db_insert_station_sensors(conn, sensor_id, sensor_parameter, station_id):
    """
    Function inserts sensors for each station to Database
    """
    sql = """INSERT INTO public.sensors (sensor_id, sensor_parameter, station_id)
                VALUES (%s, %s, %s);"""
    
    with conn.cursor() as cur:
        cur.execute(sql, (sensor_id, sensor_parameter, station_id))
    conn.commit()

In [34]:
# iterate over stations
for station in stations:
    
    station_id = station["id"]
    sensors = requests.get("http://api.gios.gov.pl/pjp-api/rest/station/sensors/{}".format(station_id)).json()
    
    # iterate over station sensors
    for sensor in sensors:
        
        sensor_id = sensor["id"]
        sensor_parameter = sensor["param"]["paramCode"]
        
        try:
            db_insert_station_sensors(conn, sensor_id, sensor_parameter, station_id)
        except Exception as e:
            print(e)
            print ("Cannot execute insertion for Sensor ID: {} (Station ID: {})".format(sensor_id, station_id))

### Create Sensors DataFrame from Sensors Table

In [55]:
sql = "SELECT * FROM public.sensors"

sensors_df = pd.read_sql_query(sql,con=conn)
sensors_df.head()

,sensor_id,sensor_parameter,station_id
0,642,NO2,114
1,644,O3,114
2,660,CO,117
3,14395,PM10,117
4,658,C6H6,117


## <font color="blue">Readings Table</font>

### Create Readings Table  

In [36]:
sql = """
    CREATE TABLE public.readings 
    (
        sensor_id INTEGER REFERENCES sensors (sensor_id),
        date VARCHAR(19) NOT NULL,
        reading FLOAT(4)
    );
"""
    
cur = conn.cursor()
cur.execute(sql)

### Create list of available Sensors

In [38]:
sql = "SELECT sensor_id FROM sensors;"

cur = conn.cursor()
cur.execute(sql)
sensors_ids = [values[0] for values in cur.fetchall()]
sensors_ids[:2]

[642, 644]

### Insert Readings Into DataBase

> <i><b>INSERT INTO**</b></i> readings(sensor_id, date, reading) <i><b>VALUES</b></i> (621, '2018-09-05 00:00:00', 32.2259));

In [39]:
def db_insert_sensor_readings(conn, sensor_id, date, reading):
    """
    Function inserts multiple readings for sensor
    """
    
    sql = """
        INSERT INTO readings (sensor_id, date, reading)
        SELECT %s, %s, %s
        WHERE
            NOT EXISTS 
            (
                SELECT * FROM readings WHERE date = %s AND sensor_id = %s
            )
    """
    
    with conn.cursor() as cur:
        cur.execute(sql, (sensor_id, date, reading, date, sensor_id))
    conn.commit()

#### Insert all available non NULL readings into Readings Table

In [40]:
for sensor_id in sensors_ids:

    data_json = requests.get("http://api.gios.gov.pl/pjp-api/rest/data/getData/{}".format(sensor_id)).json()
    data = data_json['values']
    
    # check all available data
    for row in data:

        date = row['date']
        reading = row['value']

        # if there is no value wait until next attempt
        if isinstance(reading, NoneType):
            continue

        try:
            db_insert_sensor_readings(conn, sensor_id, date, reading)
        except Exception as e:
            print(e)

### Create Readings DataFrame from Readings Table

In [56]:
sql = "SELECT * FROM public.readings"

readings_df = pd.read_sql_query(sql,con=conn)
readings_df.head()

,sensor_id,date,reading
0,642,2018-09-07 18:00:00,12.93730
1,642,2018-09-07 17:00:00,12.83960
2,642,2018-09-07 16:00:00,11.44120
3,642,2018-09-07 15:00:00,9.43838
4,642,2018-09-07 14:00:00,10.22020


In [42]:
readings_df.describe()

,sensor_id,reading
count,41096.000000,41096.000000
mean,6907.856361,64.441994
std,5898.822276,134.640051
min,50.000000,0.000000
25%,2649.000000,4.670325
50%,4777.000000,18.308200
75%,14395.000000,42.200000
max,19816.000000,2928.870000


In [43]:
readings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41096 entries, 0 to 41095
Data columns (total 3 columns):
sensor_id    41096 non-null int64
date         41096 non-null object
reading      41096 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 963.3+ KB


## Create and save GeoDataFrame to Shapefile

In [57]:
stations_df.head()

,station_id,geom
0,114,POINT (17.141125 51.115933)
1,117,POINT (17.02925 51.129378)
2,129,POINT (17.012689 51.086225)
3,52,POINT (16.180513 51.204503)
4,109,POINT (16.269677 50.768729)


In [58]:
sensors_df.head()

,sensor_id,sensor_parameter,station_id
0,642,NO2,114
1,644,O3,114
2,660,CO,117
3,14395,PM10,117
4,658,C6H6,117


In [59]:
readings_df.head()

,sensor_id,date,reading
0,642,2018-09-07 18:00:00,12.93730
1,642,2018-09-07 17:00:00,12.83960
2,642,2018-09-07 16:00:00,11.44120
3,642,2018-09-07 15:00:00,9.43838
4,642,2018-09-07 14:00:00,10.22020


In [60]:
sensors_df = sensors_df.merge(stations_df, on='station_id')
sensors_df.head()

,sensor_id,sensor_parameter,station_id,geom
0,642,NO2,114,POINT (17.141125 51.115933)
1,644,O3,114,POINT (17.141125 51.115933)
2,660,CO,117,POINT (17.02925 51.129378)
3,14395,PM10,117,POINT (17.02925 51.129378)
4,658,C6H6,117,POINT (17.02925 51.129378)


In [61]:
readings_df = gpd.GeoDataFrame(readings_df.merge(sensors_df, on='sensor_id'))
readings_df.crs = from_epsg(4326)
readings_df.rename(index=str, columns={"geom": "geometry"}, inplace=True)
readings_df.head()

,sensor_id,date,reading,sensor_parameter,station_id,geometry
0,642,2018-09-07 18:00:00,12.93730,NO2,114,POINT (17.141125 51.115933)
1,642,2018-09-07 17:00:00,12.83960,NO2,114,POINT (17.141125 51.115933)
2,642,2018-09-07 16:00:00,11.44120,NO2,114,POINT (17.141125 51.115933)
3,642,2018-09-07 15:00:00,9.43838,NO2,114,POINT (17.141125 51.115933)
4,642,2018-09-07 14:00:00,10.22020,NO2,114,POINT (17.141125 51.115933)


In [62]:
readings_df.to_file('shp/readings.shp')

In [35]:
params_df['data'] = 

,station_id,sensor_id,parameter,data,geometry
0,114,644,O3,99.860600,POINT (17.141125 51.115933)
1,117,667,O3,90.609200,POINT (17.02925 51.129378)
2,52,293,O3,87.571700,POINT (16.180513 51.204503)
3,109,616,O3,93.518300,POINT (16.269677 50.768729)
4,11,52,O3,94.954400,POINT (15.31219 50.912475)
5,38,223,O3,100.533000,POINT (16.65366 50.433493)
6,74,444,O3,101.572000,POINT (15.431719 51.31763)
7,84,493,O3,68.324000,POINT (15.739722 50.736389)
8,9153,14729,O3,93.049400,POINT (15.765608 50.913433)
9,10007,16414,O3,96.131300,POINT (16.889899 50.341864)


In [40]:
temp = (params_df['data']-params_df['data'].mean())/params_df['data'].std()
temp.head()

0    1.219786
1    0.499643
2    0.263199
3    0.726091
4    0.837880
Name: data, dtype: float64

In [41]:
temp = (temp-temp.min())/(temp.max()-temp.min())
temp.head()

0    0.709095
1    0.579090
2    0.536406
3    0.619970
4    0.640151
Name: data, dtype: float64